<a href="https://colab.research.google.com/github/przemek-c/robotyka/blob/kolokwium-v2/kinematyka_odwrotna_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://mymodelrobot.appspot.com/

In [1]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4

  Preparing metadata (setup.py) ... done
  Created wheel for colored: filename=colored-1.4.4-py3-none-any.whl size=14249 sha256=cd802cbc0a0f6c53415d0b3b60a164d0600f198ea8d08b44e2eaf8f70c0c0c2d
  Stored in directory: /root/.cache/pip/wheels/e7/c3/07/fabb0941ff5df7a487d43a67081273045536cc96d4d0e816b4
Successfully built colored
  Attempting uninstall: colored
    Found existing installation: colored 2.2.3
    Uninstalling colored-2.2.3:
      Successfully uninstalled colored-2.2.3


In [2]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

Rozwiązanie zagadnienia kinematyki odwrotnej, podobnie jak zagadnienia kinematyki prostej, trzeba rozpocząć od stworzenia modelu manipulatora.

In [3]:
L1 = rtb.DHLink(d=0.645, alpha=pi/2, theta=0.0, a=0.330,)
L2 = rtb.DHLink(d=0.0, alpha=0.0, theta=0.0, a=1.35)
L3 = rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.115)
L4= rtb.DHLink(d=1.420, alpha=-pi/2, theta=0.0, a=0.0)
L5= rtb.DHLink(d=0.0, alpha=pi/2, theta=0.0, a=0.0)
L6= rtb.DHLink(d=0.215, alpha=0.0, theta=0.0, a=0.0)
robot = rtb.DHRobot([L1, L2, L3, L4, L5, L6])

W następnym kroku konieczne jest określenie położenia i orientacji końcówki manipulatora, dla której zagadnienie  ma zostać rozwiązane. Pozycję tą należny przedstawić w postaci macierzy homogenicznej. Jednym z najprostszych sposobów jest określenie macierzy translacji i rotacji. <br>
Macierz translacji stworzymy obliczając ją dla przykładu z kinematyki prostej. W ten sposób manipulator ustawi się ostatecznie w tym samym miejscu.

Rozwiązanie rozwiązania zagadnienia kinematyki odwrotnej można otrzymać wywołując na obiekcie robota metodę **ikine_LM** i podając jako argument macierz jednorodną.
```py
sol = robot.ikine_LM(T)
```
W zwracanym obiekcie *sol* zawarte są informacje o rozwiązaniu problemu, w tym współrzędne złączowe i informacja o tym czy udało się osiągnąć zadaną pozycję.

## Kinematyka odwrotna
Kinematykę odwrotną wyznaczymy poprzez interpolację na dwa sposoby:
1. Task Space. <br>
  Interpolujemy pomiędzy dwoma punktami które określają położenie efektora w postaci (x, y, z). <br>
2. Joint Space.
  <br> Interpolujemy pomiędzy obliczonymi kątami dla dwóch różnych położeń efektora.

In [17]:
point1 = (-1, 1, 2)
point2 = (-1, -1, 2)

## Task Space:

In [5]:
def interpolate_points(point1, point2, num_points):
    interpolated_points = []
    for i in range(num_points + 1):
        t = i / num_points
        interpolated_x = point1[0] + t * (point2[0] - point1[0])
        interpolated_y = point1[1] + t * (point2[1] - point1[1])
        interpolated_z = point1[2] + t * (point2[2] - point1[2])
        interpolated_points.append((interpolated_x, interpolated_y, interpolated_z))
    return interpolated_points

num_interpolated_points = 20

points = interpolate_points(point1, point2, num_interpolated_points)

for point in points:
    print(point)


(-1.0, 1.0, 2.0)
(-1.0, 0.9, 2.0)
(-1.0, 0.8, 2.0)
(-1.0, 0.7, 2.0)
(-1.0, 0.6, 2.0)
(-1.0, 0.5, 2.0)
(-1.0, 0.4, 2.0)
(-1.0, 0.30000000000000004, 2.0)
(-1.0, 0.19999999999999996, 2.0)
(-1.0, 0.09999999999999998, 2.0)
(-1.0, 0.0, 2.0)
(-1.0, -0.10000000000000009, 2.0)
(-1.0, -0.19999999999999996, 2.0)
(-1.0, -0.30000000000000004, 2.0)
(-1.0, -0.3999999999999999, 2.0)
(-1.0, -0.5, 2.0)
(-1.0, -0.6000000000000001, 2.0)
(-1.0, -0.7, 2.0)
(-1.0, -0.8, 2.0)
(-1.0, -0.8999999999999999, 2.0)
(-1.0, -1.0, 2.0)


In [6]:
def calculate_sol(x):
  trans = SE3(x)
  y = [0,0,1]
  z = [1,0,0]
  rot = SE3.OA(y, z)
  T = trans * rot
  mask = np.array([1, 1, 1, 0, 0, 0])
  sol = robot.ikine_LM(T, mask=mask)
  if sol.success == True:
    return sol.q


In [7]:
# od zera do len(T)
qr = []
# my_time = 0.0
for i in range(len(points)):
  # my_time = my_time + 0.1
  my_sol = calculate_sol(points[i])
  # qr.append(my_time)
  qr.append(my_sol)
q_array = np.array(qr)
print(q_array)

[[   2.381  0.02733   -2.909    2.963    1.941 -0.05893]
 [   2.268    1.872  -0.2638   -2.066    1.644  -0.3631]
 [ -0.5871    1.838    2.708    1.916   -2.554    3.019]
 [   2.527   0.1686   -3.039  -0.6561    3.105   -2.969]
 [ -0.3607    1.648    2.969      1.6   -1.817   -2.881]
 [   2.676  -0.1298   -2.797 -0.01871   0.4673     1.07]
 [   2.598    2.001  -0.3932    -1.77    2.162   0.4711]
 [    2.82    2.287    -0.57    1.123  -0.1638  -0.3729]
 [   2.972    1.992  -0.4922    -2.85   -2.658   -2.678]
 [   2.904  -0.0398   -2.674    1.057  -0.8286  -0.5249]
 [ -0.1999    1.437   -3.066    -1.34   -1.249    2.775]
 [  -3.035    1.975    -0.41   -2.262   -3.099    2.905]
 [  -3.094   0.1097   -2.726   -2.336    1.771   -1.415]
 [  -2.843    2.196   -0.663   -3.103   -1.224    1.677]
 [  0.3328   -2.993  -0.2927   0.2799    1.045    -1.99]
 [  0.6278   -3.009  -0.1394   -1.624    1.018   0.8705]
 [  -2.596    0.181   -3.026   0.2116    2.998    2.662]
 [  0.5891   -2.952  -0.1798   

In [8]:
robot.plot(q_array, movie="task_space_robot_ikine.gif")

<IPython.core.display.Javascript object>

PyPlot3D backend, t = 1.0500000000000003, scene:
  robot: Text(0.0, 0.0, '')

### Creating wynik.txt

In [14]:
with open("wynik.txt", "w") as plik:
  for q in q_array:
    print(q)
    plik.write(','.join(map(str,q)))
    plik.write('\n')

[   2.381  0.02733   -2.909    2.963    1.941 -0.05893]
[   2.268    1.872  -0.2638   -2.066    1.644  -0.3631]
[ -0.5871    1.838    2.708    1.916   -2.554    3.019]
[   2.527   0.1686   -3.039  -0.6561    3.105   -2.969]
[ -0.3607    1.648    2.969      1.6   -1.817   -2.881]
[   2.676  -0.1298   -2.797 -0.01871   0.4673     1.07]
[   2.598    2.001  -0.3932    -1.77    2.162   0.4711]
[    2.82    2.287    -0.57    1.123  -0.1638  -0.3729]
[   2.972    1.992  -0.4922    -2.85   -2.658   -2.678]
[   2.904  -0.0398   -2.674    1.057  -0.8286  -0.5249]
[ -0.1999    1.437   -3.066    -1.34   -1.249    2.775]
[  -3.035    1.975    -0.41   -2.262   -3.099    2.905]
[  -3.094   0.1097   -2.726   -2.336    1.771   -1.415]
[  -2.843    2.196   -0.663   -3.103   -1.224    1.677]
[  0.3328   -2.993  -0.2927   0.2799    1.045    -1.99]
[  0.6278   -3.009  -0.1394   -1.624    1.018   0.8705]
[  -2.596    0.181   -3.026   0.2116    2.998    2.662]
[  0.5891   -2.952  -0.1798   0.2749   0.4697   

### Creating task_space_wynik_w_csv.csv

In [15]:
import pandas as pd
read_file = pd.read_csv ('wynik.txt')
read_file.to_csv ('robocze_wynik_w_csv.csv', index=None)
df = pd.read_csv('robocze_wynik_w_csv.csv')
# dodajemy naglowki
df.columns =['q1','q2','q3','q4','q5','q6']
# jaka dlugosc csv
rows_in_csv = df.shape[0]
# do takiej wartosci for tworzy liste ze skokiem 0.1
tt = []
for x in range(0, rows_in_csv):
  t = x*0.2
  tt.append(t)
# dodajemy to jako pierwsza kolumna
df['time'] = tt
df = df[['time', 'q1','q2','q3','q4','q5','q6']]
df.to_csv('task_space_wynik_w_csv.csv', index=False)

## Joint Space:

In [18]:
points = [point1, point2]

In [19]:
def calculate_sol(x):
  trans = SE3(x)
  y = [0,0,1]
  z = [1,0,0]
  rot = SE3.OA(y, z)
  T = trans * rot
  mask = np.array([1, 1, 1, 0, 0, 0])
  sol = robot.ikine_LM(T, mask=mask)
  if sol.success == True:
    return sol.q


In [21]:
# od zera do len(T)
qr = []
# my_time = 0.0
for i in range(len(points)):
  # my_time = my_time + 0.1
  my_sol = calculate_sol(points[i])
  # qr.append(my_time)
  qr.append(my_sol)
q_array = np.array(qr)
print(q_array)

[[   2.296  0.08293   -3.005   0.5867   -2.338    1.111]
 [  -2.459  0.07137   -3.024    1.038   -2.238   -2.518]]


In [24]:
def interpolate_joint_positions(joint1, joint2, num_points):
    t_values = np.linspace(0, 1, num_points + 2)
    interpolated_points = []

    for t in t_values[1:-1]:
        interpolated_joint = (1 - t) * joint1 + t * joint2
        interpolated_points.append(interpolated_joint)
    return interpolated_points

joint1 = q_array[0]
# joint1 = np.array([-0.7909, 1.665, -0.1807, -1.456, 3.105, 2.123])
print(joint1)
joint2 = q_array[1]
# joint2 = np.array([-2.367, 2.981, 0.413, 1.619, 3.067, -1.952])
num_interpolated_points = 20

points = interpolate_joint_positions(joint1, joint2, num_interpolated_points)
for point in points:
    print(point)

robot.plot(q_array, movie="joint_space_robot_ikine.gif")


[   2.296  0.08293   -3.005   0.5867   -2.338    1.111]
[   2.069  0.08238   -3.005   0.6082   -2.333   0.9378]
[   1.843  0.08183   -3.006   0.6297   -2.328   0.7649]
[   1.616  0.08128   -3.007   0.6512   -2.324   0.5921]
[    1.39  0.08073   -3.008   0.6726   -2.319   0.4193]
[   1.163  0.08018   -3.009   0.6941   -2.314   0.2465]
[   0.937  0.07963    -3.01   0.7156   -2.309  0.07369]
[  0.7106  0.07908   -3.011   0.7371   -2.305 -0.09912]
[  0.4842  0.07853   -3.012   0.7586     -2.3  -0.2719]
[  0.2578  0.07798   -3.013     0.78   -2.295  -0.4447]
[ 0.03137  0.07743   -3.014   0.8015    -2.29  -0.6176]
[ -0.1951  0.07688   -3.015    0.823   -2.285  -0.7904]
[ -0.4215  0.07632   -3.016   0.8445   -2.281  -0.9632]
[ -0.6479  0.07577   -3.017   0.8659   -2.276   -1.136]
[ -0.8743  0.07522   -3.018   0.8874   -2.271   -1.309]
[  -1.101  0.07467   -3.019   0.9089   -2.266   -1.482]
[  -1.327  0.07412    -3.02   0.9304   -2.262   -1.654]
[  -1.554  0.07357   -3.021   0.9518   -2.257   

PyPlot3D backend, t = 0.1, scene:
  robot: Text(0.0, 0.0, '')

### Creating wynik.txt

In [26]:
with open("robocze_wynik.txt", "w") as plik:
  for q in points:
    print(q)
    plik.write(','.join(map(str,q)))
    plik.write('\n')

[   2.069  0.08238   -3.005   0.6082   -2.333   0.9378]
[   1.843  0.08183   -3.006   0.6297   -2.328   0.7649]
[   1.616  0.08128   -3.007   0.6512   -2.324   0.5921]
[    1.39  0.08073   -3.008   0.6726   -2.319   0.4193]
[   1.163  0.08018   -3.009   0.6941   -2.314   0.2465]
[   0.937  0.07963    -3.01   0.7156   -2.309  0.07369]
[  0.7106  0.07908   -3.011   0.7371   -2.305 -0.09912]
[  0.4842  0.07853   -3.012   0.7586     -2.3  -0.2719]
[  0.2578  0.07798   -3.013     0.78   -2.295  -0.4447]
[ 0.03137  0.07743   -3.014   0.8015    -2.29  -0.6176]
[ -0.1951  0.07688   -3.015    0.823   -2.285  -0.7904]
[ -0.4215  0.07632   -3.016   0.8445   -2.281  -0.9632]
[ -0.6479  0.07577   -3.017   0.8659   -2.276   -1.136]
[ -0.8743  0.07522   -3.018   0.8874   -2.271   -1.309]
[  -1.101  0.07467   -3.019   0.9089   -2.266   -1.482]
[  -1.327  0.07412    -3.02   0.9304   -2.262   -1.654]
[  -1.554  0.07357   -3.021   0.9518   -2.257   -1.827]
[   -1.78  0.07302   -3.022   0.9733   -2.252   

### Creating updated_wynik_w_csv.csv

In [27]:
import pandas as pd
read_file = pd.read_csv ('wynik.txt')
read_file.to_csv ('robocze_wynik_w_csv.csv', index=None)
df = pd.read_csv('robocze_wynik_w_csv.csv')
# dodajemy naglowki
df.columns =['q1','q2','q3','q4','q5','q6']
# jaka dlugosc csv
rows_in_csv = df.shape[0]
# do takiej wartosci for tworzy liste ze skokiem 0.1
tt = []
for x in range(0, rows_in_csv):
  t = x*0.2
  tt.append(t)
# dodajemy to jako pierwsza kolumna
df['time'] = tt
df = df[['time', 'q1','q2','q3','q4','q5','q6']]
df.to_csv('joint_space_wynik_w_csv.csv', index=False)